In [6]:
import sys, os
from typing import Callable, List, Dict, Any
from langsmith import traceable, Client
import uuid
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv

load_dotenv()
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [7]:
dataset = pd.read_csv("../final_dataset.csv")

In [8]:
answers=pd.read_csv("run_20250604_181423.csv")

In [9]:
answers.head()

,run_id,input,output,reference_answer
0,431a78b6-2002-4769-95d9-8056b4a969c7,how many draft orders (brouillon) we have?,"{'output': ""There are **0 draft orders** (labe...",We have 40 draft orders (brouillon).
1,84078d88-05e8-481c-aec7-32e8d5188e6a,how many transport orders we have in total ?,{'output': 'The total number of transport orde...,"We have a total of 65,776 transport orders."
2,182e75f8-f66e-4729-b253-701e9515e277,total number of orders that have planned loadi...,"{'output': 'There are **31,212 orders** with a...","31,212 orders have a planned loading date."
3,ca106dc5-ef7c-4a9d-9399-46b95fa2b2da,orders count that was handled by external agency?,"{'output': 'There are **24,214 orders** that w...","24,214 orders were handled by external agencies."
4,5e4d816d-a6dc-4bc8-9b7e-0dfb973477ba,give me number of orders where the dock suppor...,"{'output': 'There are **34,877 orders** where ...","34,877 orders utilize dock supports for pallets."


In [10]:
results=pd.merge(answers, dataset, left_on="input", right_on="question", how="left")

In [11]:
results

,run_id,input,output,reference_answer,question,answer
0,431a78b6-2002-4769-95d9-8056b4a969c7,how many draft orders (brouillon) we have?,"{'output': ""There are **0 draft orders** (labe...",We have 40 draft orders (brouillon).,how many draft orders (brouillon) we have?,We have 40 draft orders (brouillon).
1,84078d88-05e8-481c-aec7-32e8d5188e6a,how many transport orders we have in total ?,{'output': 'The total number of transport orde...,"We have a total of 65,776 transport orders.",how many transport orders we have in total ?,"We have a total of 65,776 transport orders."
2,182e75f8-f66e-4729-b253-701e9515e277,total number of orders that have planned loadi...,"{'output': 'There are **31,212 orders** with a...","31,212 orders have a planned loading date.",total number of orders that have planned loadi...,"31,212 orders have a planned loading date."
3,ca106dc5-ef7c-4a9d-9399-46b95fa2b2da,orders count that was handled by external agency?,"{'output': 'There are **24,214 orders** that w...","24,214 orders were handled by external agencies.",orders count that was handled by external agency?,"24,214 orders were handled by external agencies."
4,5e4d816d-a6dc-4bc8-9b7e-0dfb973477ba,give me number of orders where the dock suppor...,"{'output': 'There are **34,877 orders** where ...","34,877 orders utilize dock supports for pallets.",give me number of orders where the dock suppor...,"34,877 orders utilize dock supports for pallets."
...,...,...,...,...,...,...
115,06a0b251-a9b7-4909-96d6-8de0c791ca10,total number of orders that contain “Chocolat”...,"{'output': ""After analyzing the order data fro...","No product lines contain the word ""Chocolat"". ...",total number of orders that contain “Chocolat”...,"No product lines contain the word ""Chocolat"". ..."
116,5543d6de-838c-4366-a4ea-2056154ecfe3,average quantity for orders going to “Tamanras...,"{'output': ""No orders with the destination 'Ta...",“Tamanrasset” is not a destination found in th...,average quantity for orders going to “Tamanras...,“Tamanrasset” is not a destination found in th...
117,cefbb3df-1a84-4391-83b8-47b5701d4d50,give me the number of docks with MainGroup_Doc...,{'output': 'The task could not be completed du...,“QXYZ” is not a valid value in the MainGroup_D...,give me the number of docks with MainGroup_Doc...,“QXYZ” is not a valid value in the MainGroup_D...
118,dea18131-c973-420b-ace6-2f3e892de0bc,number of docks handling “Lait” (milk) products,"{'output': ""After analyzing the order data fro...",No product entries contain the keyword “Lait”....,number of docks handling “Lait” (milk) products,No product entries contain the keyword “Lait”....


In [12]:
import ast

results[["generated_answer", "plan"]] = results["output"].apply(
    lambda x: pd.Series(ast.literal_eval(x)) if isinstance(x, str) and x.startswith("{") else pd.Series({"output": None, "plan": None})
)[["output", "plan"]]

In [13]:
results.drop(columns=["output","question","answer"], inplace=True)

In [14]:
results

,run_id,input,reference_answer,generated_answer,plan
0,431a78b6-2002-4769-95d9-8056b4a969c7,how many draft orders (brouillon) we have?,We have 40 draft orders (brouillon).,There are **0 draft orders** (labeled as 'brou...,1. Loaded order data from 2024-12-01 to 2025-0...
1,84078d88-05e8-481c-aec7-32e8d5188e6a,how many transport orders we have in total ?,"We have a total of 65,776 transport orders.",The total number of transport orders in the da...,1. Loaded order data using data_loader for the...
2,182e75f8-f66e-4729-b253-701e9515e277,total number of orders that have planned loadi...,"31,212 orders have a planned loading date.","There are **31,212 orders** with a planned loa...",1. Loaded order data for the entire date range...
3,ca106dc5-ef7c-4a9d-9399-46b95fa2b2da,orders count that was handled by external agency?,"24,214 orders were handled by external agencies.","There are **24,214 orders** that were handled ...",1. Loaded order data from 2024-12-01 to 2025-0...
4,5e4d816d-a6dc-4bc8-9b7e-0dfb973477ba,give me number of orders where the dock suppor...,"34,877 orders utilize dock supports for pallets.","There are **34,877 orders** where the dock sup...",1. Loaded order data from 2024-12-01 to 2025-0...
...,...,...,...,...,...
115,06a0b251-a9b7-4909-96d6-8de0c791ca10,total number of orders that contain “Chocolat”...,"No product lines contain the word ""Chocolat"". ...",After analyzing the order data from 2024-12-01...,1. Loaded order data for the date range using ...
116,5543d6de-838c-4366-a4ea-2056154ecfe3,average quantity for orders going to “Tamanras...,“Tamanrasset” is not a destination found in th...,No orders with the destination 'Tamanrasset' w...,1. Loaded order data for the date range 2024-1...
117,cefbb3df-1a84-4391-83b8-47b5701d4d50,give me the number of docks with MainGroup_Doc...,“QXYZ” is not a valid value in the MainGroup_D...,The task could not be completed due to time or...,Review the task parameters and adjust the time...
118,dea18131-c973-420b-ace6-2f3e892de0bc,number of docks handling “Lait” (milk) products,No product entries contain the keyword “Lait”....,After analyzing the order data from December 1...,1. Loaded order data from 2024-12-01 to 2025-0...


In [15]:
langsmith_client = Client()


In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() - timedelta(hours=2)

In [55]:
start_date.strftime("%Y-%m-%d %H:%M:%S")

'2025-06-08 01:00:10'

In [56]:
runs=list(langsmith_client.list_runs(project_name="PFE",start_time=start_date, execution_order=1))

In [57]:
len(runs)


102

In [ ]:
def get_langsmith_data(results: List[Dict[str, Any]]) -> pd.DataFrame:
    run_ids = {r["run_id"] for r in results}  # Use set for faster lookup
    print(f"📡 Fetching LangSmith run data for {len(run_ids)} runs...\n")
    start_date = datetime.now() - timedelta(hours=2)
    runs = list(langsmith_client.list_runs(project_name="PFE", execution_order=1,  start_time=start_date  ))
    print(f"📊 Found {len(runs)} runs in LangSmith...\n")
    data = []
    run_id_to_r = {r["run_id"]: r for r in results}
    for run in runs:
        if run.name in run_id_to_r:
            r = run_id_to_r[run.name]
            data.append({
                "run_id": run.id,
                "duration (sec)": (run.end_time - run.start_time).total_seconds()
                if run.end_time and run.start_time else None,
                "error": run.error,
                "total_tokens": run.total_tokens,
                "question": r.get("question"),
                "generated_answer": r.get("generated_answer"),
            })
    print(f"📊 Assembling results into DataFrame...\n")
    df = pd.DataFrame(data)
    return df

In [91]:
results=pd.read_csv("../../scripts/runs/lnGxEDxx.csv").to_dict(orient="records")

In [ ]:
res=get_langsmith_data(results=results)

In [ ]:
res

In [94]:
res.to_csv("langsmith_data.csv", index=False)